In [1]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate, Lambda, Add
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras import backend as K
from keras.preprocessing.text import Tokenizer
import random

Using TensorFlow backend.


In [3]:
df = pd.concat([
    pd.read_csv('../input/poems/haikus_with_syllables.csv', encoding='latin1'),
    pd.read_csv('../input/poems/haikus_sballas8_with_syllables.csv', encoding='latin1'),
    pd.read_csv('../input/poems/haiku_unim.csv', encoding='latin1')
]).reset_index(drop=True)
df = df[df['lang'] == 'en'].copy()
df = df[~pd.isnull(df['0']) & ~pd.isnull(df['1']) & ~pd.isnull(df['2'])].copy()
df = df[~pd.isnull(df['0_syllables']) & ~pd.isnull(df['1_syllables']) & ~pd.isnull(df['2_syllables'])].copy()

# Duplicate lines with ambiguous syllable counts
lines = set([0, 1, 2])

for i in range(3):
    lines.remove(i)
    df = df[[
        '0', '1', '2',
        #'1_syllables', '2_syllables'
    ] + ['%s_syllables' % j for j in lines]].join(
        df['%s_syllables' % i].str.split(
            ',', expand=True
        ).stack(-1).reset_index(
            level=1, drop=True
        ).rename('%s_syllables' % i)
    ).drop_duplicates()
    lines.add(i)

df

/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,0,1,2,0_syllables,1_syllables,2_syllables
0,Memorial Day --,a shadow for each,white cross,5,5,2
1,spring rain -,as the doctor speaks,i think of lilacs,2,5,5
1,spring rain -,as the doctor speaks,i think of lilacs,3,5,5
2,spring moonset --,a rice ball for,breakfast,3,4,2
2,spring moonset --,a rice ball for,breakfast,4,4,2
3,sunny afternoon,an old man lingers,near the mailbox,5,5,4
4,cinco de mayo,horses roll,in the shallows,5,3,4
5,quitting time,the smell of rain,in the lobby,3,4,4
6,waves,slowly cresting towards shore,a faint moon,1,6,3
6,waves,slowly cresting towards shore,a faint moon,1,7,3


In [4]:
max_line_len = int(max([df['%s' % i].str.len().quantile(.99) for i in range(3)]))
df = df[
    (df['0'].str.len() <= max_line_len) & 
    (df['1'].str.len() <= max_line_len) & 
    (df['2'].str.len() <= max_line_len)
]

for i in range(3):
    #i = str(i)
    df['%s_in' % i] = df[str(i)].str[0] + df[str(i)].str.pad(max_line_len, 'right', '\n')
    df['%s_out' % i] = df[str(i)].str.pad(max_line_len, 'right', '\n') + ('\n' if i == 2 else df[str(i+1)].str[0])
    #df['%s_out' % i] = df[str(i)] + ('\n' if i == 2 else df[str(i+1)].str[0])
    #df['%s_out' % i] = df['%s_out' % i].str.pad(max_line_len + 1, 'right', '\n')
    
max_line_len += 1

#df

/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [5]:
inputs = df[['0_in', '1_in', '2_in']].values

t = Tokenizer(filters='', char_level=True)
t.fit_on_texts(inputs.flatten())
n_tokens = len(t.word_counts) + 1

X = np_utils.to_categorical([
    t.texts_to_sequences(inputs[:,i]) for i in range(3)
], num_classes=n_tokens)

outputs = df[['0_out', '1_out', '2_out']].values

#t = Tokenizer(filters='', char_level=True)
#t.fit_on_texts(outputs.flatten())
Y = np_utils.to_categorical([
    t.texts_to_sequences(outputs[:,i]) for i in range(3)
], num_classes=n_tokens)

n_tokens = len(t.word_counts) + 1

X_syllables = df[['0_syllables', '1_syllables', '2_syllables']].values

In [6]:
latent_dim = 512

# Keras Model

inputs = [Input(shape=(None, n_tokens), name='input_line_%s' % i) for i in range(3)]

syllables = [Input(shape=(1,), name='syllables_line_%s' % i) for i in range(3)]
syllable_denses = [Dense(latent_dim, activation='relu', name='syllable_dense_%s' % i) for i in range(3)]
syllable_dense_outputs = [syllable_denses[i](syllables[i]) for i in range(3)]

lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, name='lstm_%s' % i) for i in range(3)]

lstm_out, lstm_h, lstm_c = [None, None, None], [None, None, None], [None, None, None]
for i in range(3):
    if i > 0:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[
            Add(name='add_h_%s' % i)([
                lstm_h[i-1],
                syllable_dense_outputs[i]
            ]),
            Add(name='add_c_%s' % i)([
                lstm_c[i-1],
                syllable_dense_outputs[i]
            ])
        ])
    else:
        lstm_out[i], lstm_h[i], lstm_c[i] = lstms[i](inputs[i], initial_state=[
            syllable_dense_outputs[i], syllable_dense_outputs[i]
        ])
        
denses = [Dense(n_tokens, activation='softmax', name='output_%s' % i) for i in range(3)]
outputs = [denses[i](lstm_out[i]) for i in range(3)]

# Enforce number of chars per line
#counter = Lambda(lambda x: K.sum(K.cast(K.less(K.argmax(x, axis=2), 60), K.floatx()), axis=1, keepdims=True))
#counts = [counter(i) for i in outputs]

#model = Model(inputs, outputs + counts)

model = Model(inputs + syllables, outputs)

# Use categorical for the character outputs and MSE for the counts
#model.compile(optimizer='rmsprop', loss=['categorical_crossentropy']*3 + ['mean_squared_error']*3)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
syllables_line_0 (InputLayer)                                     (None, 1)                                   0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
input_line_0 (InputLayer)                                         (None, None, 64)                            0                                                                                     

In [6]:
np.argmax(X[0][1], axis=1)

array([ 7,  7, 21, 10,  9,  8, 17,  2, 10,  5,  9,  8,  2, 25,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1], dtype=int64)

In [42]:
model.fit([
    X[0], X[1], X[2],
    X_syllables[:,0], X_syllables[:,1], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=64, epochs=1000)

Epoch 1/1000
24777/24777 [==============================] - 344s 14ms/step - loss: 1.3306 - output_0_loss: 0.4516 - output_1_loss: 0.5081 - output_2_loss: 0.3709
Epoch 2/1000
24777/24777 [==============================] - 416s 17ms/step - loss: 1.2727 - output_0_loss: 0.4332 - output_1_loss: 0.4883 - output_2_loss: 0.3511
Epoch 3/1000
24777/24777 [==============================] - 436s 18ms/step - loss: 1.2187 - output_0_loss: 0.4165 - output_1_loss: 0.4701 - output_2_loss: 0.3322
Epoch 4/1000
24777/24777 [==============================] - 472s 19ms/step - loss: 1.1673 - output_0_loss: 0.3999 - output_1_loss: 0.4530 - output_2_loss: 0.3144
Epoch 5/1000
24777/24777 [==============================] - 448s 18ms/step - loss: 1.1179 - output_0_loss: 0.3842 - output_1_loss: 0.4364 - output_2_loss: 0.2973
Epoch 6/1000
24777/24777 [==============================] - 446s 18ms/step - loss: 1.0719 - output_0_loss: 0.3693 - output_1_loss: 0.4207 - output_2_loss: 0.2820
Epoch 7/1000
24777/24777 [==

24777/24777 [==============================] - 433s 17ms/step - loss: 0.4528 - output_0_loss: 0.1805 - output_1_loss: 0.1854 - output_2_loss: 0.0869
Epoch 52/1000
24777/24777 [==============================] - 431s 17ms/step - loss: 0.4499 - output_0_loss: 0.1801 - output_1_loss: 0.1841 - output_2_loss: 0.0857
Epoch 53/1000
24777/24777 [==============================] - 450s 18ms/step - loss: 0.4455 - output_0_loss: 0.1790 - output_1_loss: 0.1822 - output_2_loss: 0.0843
Epoch 54/1000
24777/24777 [==============================] - 428s 17ms/step - loss: 0.4423 - output_0_loss: 0.1779 - output_1_loss: 0.1809 - output_2_loss: 0.0834
Epoch 55/1000
24777/24777 [==============================] - 445s 18ms/step - loss: 0.4398 - output_0_loss: 0.1772 - output_1_loss: 0.1796 - output_2_loss: 0.0830
Epoch 56/1000
24777/24777 [==============================] - 434s 18ms/step - loss: 0.4360 - output_0_loss: 0.1767 - output_1_loss: 0.1779 - output_2_loss: 0.0814
Epoch 57/1000
24777/24777 [=========

KeyboardInterrupt: 

In [71]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
model.fit([
    X[0], X[1], X[2],
    X_syllables[:,0], X_syllables[:,1], X_syllables[:,2]
], [Y[0], Y[1], Y[2]], batch_size=64, epochs=1000, callbacks=callbacks_list)

Epoch 1/1000
24777/24777 [==============================] - 354s 14ms/step - loss: 0.3664 - output_0_loss: 0.1586 - output_1_loss: 0.1469 - output_2_loss: 0.0608

Epoch 00001: loss improved from -inf to 0.36639, saving model to weights-improvement-01-0.37.hdf5


/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_0 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'syllable_dense_0/Relu:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'syllable_dense_0/Relu:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'add_h_1/add:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'add_c_1/add:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarn

Epoch 2/1000
24777/24777 [==============================] - 421s 17ms/step - loss: 0.3633 - output_0_loss: 0.1576 - output_1_loss: 0.1456 - output_2_loss: 0.0601

Epoch 00002: loss did not improve from 0.36639
Epoch 3/1000
24777/24777 [==============================] - 428s 17ms/step - loss: 0.3621 - output_0_loss: 0.1573 - output_1_loss: 0.1452 - output_2_loss: 0.0596

Epoch 00003: loss did not improve from 0.36639
Epoch 4/1000
24777/24777 [==============================] - 437s 18ms/step - loss: 0.3609 - output_0_loss: 0.1571 - output_1_loss: 0.1440 - output_2_loss: 0.0597

Epoch 00004: loss did not improve from 0.36639
Epoch 5/1000
24777/24777 [==============================] - 431s 17ms/step - loss: 0.3589 - output_0_loss: 0.1565 - output_1_loss: 0.1435 - output_2_loss: 0.0589

Epoch 00005: loss did not improve from 0.36639
Epoch 6/1000
24777/24777 [==============================] - 454s 18ms/step - loss: 0.3574 - output_0_loss: 0.1562 - output_1_loss: 0.1428 - output_2_loss: 0.058

24777/24777 [==============================] - 444s 18ms/step - loss: 0.3136 - output_0_loss: 0.1447 - output_1_loss: 0.1212 - output_2_loss: 0.0478

Epoch 00041: loss did not improve from 0.36639
Epoch 42/1000
24777/24777 [==============================] - 438s 18ms/step - loss: 0.3127 - output_0_loss: 0.1444 - output_1_loss: 0.1205 - output_2_loss: 0.0478

Epoch 00042: loss did not improve from 0.36639
Epoch 43/1000
24777/24777 [==============================] - 460s 19ms/step - loss: 0.3117 - output_0_loss: 0.1440 - output_1_loss: 0.1203 - output_2_loss: 0.0474

Epoch 00043: loss did not improve from 0.36639
Epoch 44/1000
24777/24777 [==============================] - 467s 19ms/step - loss: 0.3100 - output_0_loss: 0.1440 - output_1_loss: 0.1195 - output_2_loss: 0.0465

Epoch 00044: loss did not improve from 0.36639
Epoch 45/1000
24777/24777 [==============================] - 436s 18ms/step - loss: 0.3079 - output_0_loss: 0.1435 - output_1_loss: 0.1180 - output_2_loss: 0.0464

Epoch 

KeyboardInterrupt: 

In [86]:
model.save('model.hdf5')

/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_0 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'syllable_dense_0/Relu:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'syllable_dense_0/Relu:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'add_h_1/add:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'add_c_1/add:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarn

In [72]:
generator_inputs = [Input(shape=(None, n_tokens), name='generator_input_%s' % i) for i in range(3)]
generator_inputs_h = [Input(shape=(latent_dim,), name='generator_in_h_%s' % i) for i in range(3)]
generator_inputs_c = [Input(shape=(latent_dim,), name='generator_in_c_%s' % i) for i in range(3)]
generator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, name='generator_lstm_%s' % i) for i in range(3)]
generator_lstm_out, generator_lstm_h, generator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
generator_denses = [Dense(n_tokens, activation='softmax') for i in range(3)]

generator_syllables = [Input(shape=(1,), name='generator_syllables_%s' % i) for i in range(3)]
generator_syllable_denses = [Dense(latent_dim, activation='relu', name='generator_syllable_dense_%s' % i) for i in range(3)]
#syllable_dense_outputs = [syllable_denses[i](syllables[i]) for i in range(3)]

generator_syllable_dense_outputs = []
generator_outputs = []
generator_models = []

for i in range(3):
    generator_syllable_dense_outputs.append(generator_syllable_denses[i](generator_syllables[i]))
    generator_lstm_out[i], generator_lstm_h[i], generator_lstm_c[i] = generator_lstms[i](
        generator_inputs[i], initial_state=[
            Add(name='generator_add_h_%s' % i)([
                generator_inputs_h[i],
                generator_syllable_dense_outputs[i]
            ]),
            Add(name='generator_add_c_%s' % i)([
                generator_inputs_c[i],
                generator_syllable_dense_outputs[i]
            ])
        ]
    )
    generator_outputs.append(generator_denses[i](generator_lstm_out[i]))

    generator_models.append(Model(
        [generator_inputs[i], generator_syllables[i], generator_inputs_h[i], generator_inputs_c[i]],
        [generator_outputs[i], generator_lstm_h[i], generator_lstm_c[i]]
    ))

    generator_syllable_denses[i].set_weights(syllable_denses[i].get_weights())
    generator_lstms[i].set_weights(lstms[i].get_weights())
    generator_denses[i].set_weights(denses[i].get_weights())

'''# Hook up line 0
generator_syllable_dense_outputs = [generator_syllable_denses[0](generator_syllables[0])]
generator_lstm_out[0], generator_lstm_h[0], generator_lstm_c[0] = generator_lstms[0](
    generator_inputs[0], initial_state=[generator_syllable_dense_outputs[0], generator_syllable_dense_outputs[0]]
)
generator_outputs = [generator_denses[0](generator_lstm_out[0])]
generator_models = [Model([generator_inputs[0], generator_syllables[0]], [generator_outputs[0], generator_lstm_h[0], generator_lstm_c[0]])]

# Set weights for line 0
generator_syllable_denses[0].set_weights(syllable_denses[0].get_weights())
generator_lstms[0].set_weights(lstms[0].get_weights())
generator_denses[0].set_weights(denses[0].get_weights())'''

generator_models[0].summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
generator_syllables_0 (InputLay (None, 1)            0                                            
__________________________________________________________________________________________________
generator_in_h_0 (InputLayer)   (None, 512)          0                                            
__________________________________________________________________________________________________
generator_syllable_dense_0 (Den (None, 512)          1024        generator_syllables_0[0][0]      
__________________________________________________________________________________________________
generator_in_c_0 (InputLayer)   (None, 512)          0                                            
__________________________________________________________________________________________________
generator_

In [73]:
fgenerator_inputs = [Input(shape=(None, n_tokens), name='fgenerator_input_%s' % i) for i in range(3)]
fgenerator_inputs_h = [Input(shape=(latent_dim,), name='fgenerator_in_h_%s' % i) for i in range(3)]
fgenerator_inputs_c = [Input(shape=(latent_dim,), name='fgenerator_in_c_%s' % i) for i in range(3)]
fgenerator_lstms = [LSTM(latent_dim, return_state=True, return_sequences=True, name='fgenerator_lstm_%s' % i) for i in range(3)]
fgenerator_lstm_out, fgenerator_lstm_h, fgenerator_lstm_c = [None, None, None], [None, None, None], [None, None, None]
fgenerator_denses = [Dense(n_tokens, activation='softmax') for i in range(3)]

#fgenerator_syllables = [Input(shape=(1,), name='fgenerator_syllables_%s' % i) for i in range(3)]
#fgenerator_syllable_denses = [Dense(latent_dim, activation='relu', name='fgenerator_syllable_dense_%s' % i) for i in range(3)]

#fgenerator_syllable_dense_outputs = []
fgenerator_outputs = []
fgenerator_models = []

for i in range(3):
    #fgenerator_syllable_dense_outputs.append(fgenerator_syllable_denses[i](fgenerator_syllables[i]))
    fgenerator_lstm_out[i], fgenerator_lstm_h[i], fgenerator_lstm_c[i] = fgenerator_lstms[i](
        fgenerator_inputs[i], initial_state=[fgenerator_inputs_h[i], fgenerator_inputs_c[i]]
        
    )
    fgenerator_outputs.append(fgenerator_denses[i](fgenerator_lstm_out[i]))

    fgenerator_models.append(Model(
        [fgenerator_inputs[i], fgenerator_inputs_h[i], fgenerator_inputs_c[i]],
        [fgenerator_outputs[i], fgenerator_lstm_h[i], fgenerator_lstm_c[i]]
    ))

    #generator_syllable_denses[i].set_weights(syllable_denses[i].get_weights())
    fgenerator_lstms[i].set_weights(lstms[i].get_weights())
    fgenerator_denses[i].set_weights(denses[i].get_weights())

fgenerator_models[0].summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
fgenerator_input_0 (InputLayer) (None, None, 64)     0                                            
__________________________________________________________________________________________________
fgenerator_in_h_0 (InputLayer)  (None, 512)          0                                            
__________________________________________________________________________________________________
fgenerator_in_c_0 (InputLayer)  (None, 512)          0                                            
__________________________________________________________________________________________________
fgenerator_lstm_0 (LSTM)        [(None, None, 512),  1181696     fgenerator_input_0[0][0]         
                                                                 fgenerator_in_h_0[0][0]          
          

In [85]:
fgenerator_models[0].save('fgenerator_model0.hdf5')

/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer fgenerator_lstm_0 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'fgenerator_in_h_0_2:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'fgenerator_in_c_0_2:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [78]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

h,c = np.zeros(latent_dim).reshape((1,-1)), np.zeros(latent_dim).reshape((1,-1))
#first = 'maw'
pattern = [5,7,5]
#generator_models[0].reset_states()
#generator_lstms[0].reset_states()
for i,syllables in enumerate(pattern):
    #line = [random.randint(2,21)]
    if i == 0:
        line = [random.randint(2,21)]
    else:
        if 1 in line:
            line = [line[line.index(1)-1]]
        else:
            line = [line[-1]]
    #line = t.texts_to_sequences(first[i])[0]
    #print(line)
    
    ## FIRST CHAR
    def f():
        while True:
            yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), np.array([syllables]), h, c]
    
    character, h, c = generator_models[i].predict_generator(f(), steps=1)    
    line.append(np.argmax(character))
    
    ## ALL THE REST (Don't include syllables, use fgenerator)
    def f():
        while True:
            yield [np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)), h, c]
            
    # While less than the max length and the last character isn't the line terminator
    while (len(line) < max_line_len):# and (line[-1] != 1):
        character, h, c = fgenerator_models[i].predict_generator(f(), steps=1)
        #line.append(np.argmax(character))
        
        
        line.append(sample(character[0][0], .3))
        #print (np.argmax(character), line[-1])
        
    #print(line[1:line.index(1)])
    #print(line)
    text = t.sequences_to_texts([line])[0].strip()[1:].replace('   ', '\n').replace(' ', '').replace('\n', ' ')
    #print(text[1:])
    print(text)
    #if 1 in line:
    #    #print(t.sequences_to_texts([line[1:line.index(1)]]))
    #else:
    #    #print(t.sequences_to_texts([line[1:]]))

parking lot at dawn?
jumpiding despite on                                               a
never sallaty


/home/docmario/miniconda3/envs/haiku/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [15]:
line

[4,
 4,
 11,
 3,
 2,
 22,
 10,
 3,
 5,
 23,
 2,
 5,
 8,
 13,
 2,
 4,
 11,
 3,
 2,
 7,
 15,
 8,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [32]:
line[line.index(1)-1]

12

In [25]:
sorted(list(t.word_counts.items()), key=lambda x: -x[1])

[('\n', 1176567),
 (' ', 91088),
 ('e', 70913),
 ('t', 56983),
 ('a', 49723),
 ('s', 49409),
 ('o', 47840),
 ('n', 45498),
 ('i', 44011),
 ('r', 41508),
 ('h', 34613),
 ('l', 29824),
 ('d', 22763),
 ('m', 18905),
 ('g', 18866),
 ('f', 17798),
 ('c', 17475),
 ('u', 16137),
 ('w', 16076),
 ('p', 12694),
 ('b', 11695),
 ('y', 10017),
 ('k', 6938),
 ('-', 4682),
 ('v', 4642),
 ('.', 4179),
 ("'", 2211),
 ('?', 1459),
 ('z', 964),
 ('j', 822),
 ('q', 540),
 ('x', 503),
 (',', 317),
 (':', 122),
 ('"', 67),
 ('~', 61),
 ('!', 59),
 ('1', 57),
 (';', 47),
 ('2', 45),
 ('>', 38),
 ('0', 35),
 ('<', 30),
 ('3', 28),
 ('&', 21),
 ('4', 19),
 ('7', 17),
 ('6', 14),
 ('5', 14),
 ('9', 14),
 ('8', 13),
 ('é', 6),
 ('*', 5),
 ('/', 5),
 ('(', 3),
 ('\xa0', 2),
 ('[', 2),
 ('=', 1),
 ('ä', 1),
 (']', 1),
 (')', 1)]

In [23]:
list(t.word_counts)

['m',
 'e',
 'o',
 'r',
 'i',
 'a',
 'l',
 ' ',
 'd',
 'y',
 '-',
 '\n',
 's',
 'h',
 'w',
 'f',
 'c',
 't',
 'p',
 'n',
 'g',
 'k',
 'b',
 'u',
 'x',
 'q',
 'v',
 'z',
 '~',
 "'",
 '.',
 'j',
 ',',
 '!',
 '<',
 '>',
 '?',
 ':',
 '2',
 '1',
 '*',
 '7',
 '3',
 '0',
 ';',
 '6',
 '5',
 '8',
 '&',
 'é',
 '"',
 '9',
 '4',
 '(',
 '=',
 'ä',
 '\xa0',
 '[',
 ']',
 '/',
 ')']